# 本库的基本用法

## 目前主要包含以下几个功能

- 计算原子温度
- 计算原子结构熵
- 计算中心对称参数
- 建立简单晶体模型
- 建立原子近邻列表

## 导入必要的package

In [1]:
import mdapy as mp # 无特殊说明本库简写为 mp
import numpy as np
import taichi as ti
ti.init(arch=ti.cpu) #arch=ti.cpu 使用CPU. # arch=ti.gpu, device_memory_GB=5. 使用GPU

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, win, python 3.7.3
[Taichi] Starting on arch=x64


## 查看版本号

In [2]:
mp.__version__

'0.4.0'

## 读取一个系统
[可在此处下载example.dump文件](http://yunpan.mushroomfire.com/s/GzTv)

In [3]:
%%time
system = mp.System('example.dump') 

Wall time: 2.96 s


## 查看系统数据

In [4]:
system.data

,id,type,x,y,z,vx,vy,vz,c_1,c_2
0,192295,3,-27.9096,42.0121,-22.2491,-2.35674,0.481940,-0.257686,5.06650,-3.71151
1,255932,5,-30.0338,43.2841,-22.1938,1.55510,-1.125920,-1.983760,5.16323,-3.69935
2,319374,2,-29.4179,42.0487,-20.1799,-1.56154,-0.105773,-1.052900,6.02419,-3.51732
3,383068,3,-27.3053,43.2945,-20.1681,1.75865,0.962938,-2.273980,5.15436,-3.90756
4,700644,2,-27.9671,44.5056,-22.1799,2.11749,-0.684720,-0.795077,6.15314,-3.85788
...,...,...,...,...,...,...,...,...,...,...
2287143,1715363,1,329.5120,257.9240,323.0480,1.93039,1.215740,-1.869160,5.89311,-3.92515
2287144,1905765,5,327.2950,256.6070,318.9410,2.16960,-4.590730,1.395140,4.94041,-4.08289
2287145,1966370,3,329.5330,257.9350,318.9310,2.65692,-3.539740,-1.125020,5.70667,-3.83374
2287146,2030121,1,328.9320,256.6390,321.0620,1.25297,0.868006,1.104770,5.42057,-3.39951


## 生成邻域列表

In [5]:
%%time
system.build_neighbor(rc=5., max_neigh=60)

Wall time: 3.82 s


## 计算中心对称参数

In [6]:
%%time
system.cal_centro_symmetry_parameter(N=12)

Wall time: 8.7 s


## 计算原子构型熵

In [7]:
%%time
system.cal_atomic_entropy()

Wall time: 4.31 s


## 计算平均原子温度

In [8]:
%%time
system.cal_atomic_temperature(amass=np.array([58.933, 58.693, 55.847, 26.982, 63.546]))

Wall time: 1.21 s


## 查看计算结果

In [9]:
system.data

,id,type,x,y,z,vx,vy,vz,c_1,c_2,csp,atomic_entropy,atomic_temp
0,192295,3,-27.9096,42.0121,-22.2491,-2.35674,0.481940,-0.257686,5.06650,-3.71151,5.066506,-3.704394,271.311096
1,255932,5,-30.0338,43.2841,-22.1938,1.55510,-1.125920,-1.983760,5.16323,-3.69935,5.163225,-3.693706,244.521713
2,319374,2,-29.4179,42.0487,-20.1799,-1.56154,-0.105773,-1.052900,6.02419,-3.51732,6.024189,-3.511253,240.937302
3,383068,3,-27.3053,43.2945,-20.1681,1.75865,0.962938,-2.273980,5.15436,-3.90756,5.154361,-3.897303,230.167953
4,700644,2,-27.9671,44.5056,-22.1799,2.11749,-0.684720,-0.795077,6.15314,-3.85788,6.153219,-3.849362,243.981369
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287143,1715363,1,329.5120,257.9240,323.0480,1.93039,1.215740,-1.869160,5.89311,-3.92515,5.893107,-3.916707,260.688812
2287144,1905765,5,327.2950,256.6070,318.9410,2.16960,-4.590730,1.395140,4.94041,-4.08289,4.940413,-4.073879,320.674103
2287145,1966370,3,329.5330,257.9350,318.9310,2.65692,-3.539740,-1.125020,5.70667,-3.83374,5.706692,-3.825464,344.000671
2287146,2030121,1,328.9320,256.6390,321.0620,1.25297,0.868006,1.104770,5.42057,-3.39951,5.420572,-3.393086,241.169174


## 查看平均温度

In [10]:
system.data['atomic_temp'].mean() # K

293.87168742301066

## 与LAMMPS计算结果进行对比

In [11]:
# csp 对比
error = abs(system.data['c_1'] - system.data['csp'])
error[error>0.1], error.max()

(Series([], dtype: float64), 0.0049196543884280075)

In [12]:
# 构型熵对比, 这里由于中间数值积分的地方我不清楚lammps的具体方式,结果数值上会有一些区别。
error = abs(system.data['c_2'] - system.data['atomic_entropy'])
error[error>0.1], error.max()

(Series([], dtype: float64), 0.08606090263366717)

## 保存计算结果到dump文件中

In [13]:
%%time
system.write_dump() # 生成一个新的dump文件

Wall time: 30.1 s


## 建立简单晶体结构

In [14]:
%%time
FCC = mp.LatticeMaker(4.05, 'FCC', 50, 50, 50)
FCC.compute()

Wall time: 506 ms


## 保存为data格式

In [15]:
%%time
FCC.write_data()

Wall time: 3.23 s
